In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/FinalProject/

/content/drive/My Drive/FinalProject


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.8/702.8 kB 14.2 MB/s eta 0:00:00


In [ ]:
import cv2

from ultralytics import YOLO
import os
from IPython.display import Image, display
from IPython import display
display.clear_output()
!yolo mode=checks

#model = YOLO("yolov8n.pt")

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 508, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.10/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are ('train', 'val', 'predict', 'export', 'track', 'benchmark').

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose', 'obb')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at ht

# Dataset pre-processing

In [ ]:
def convert_to_yolov8_format(bbox, image_width, image_height):
    x_center = (bbox[0] + bbox[2] / 2) / image_width
    y_center = (bbox[1] + bbox[3] / 2) / image_height
    width = bbox[2] / image_width
    height = bbox[3] / image_height

    return x_center, y_center, width, height

In [ ]:
import json

desired_category_ids = [4, 6, 15]
category_limits = {4: 150, 6:100, 15: 300}

dataset_dir = '/content/drive/MyDrive/FinalProject/dataset/labeling/TL_PM_PM_sedan_3'
dst_dir = '/content/drive/MyDrive/FinalProject/dataset'

# desired_category_ids 의 초기 카운트를 0으로 설정
category_counts = {category_id: 0 for category_id in desired_category_ids}

# Iterate through all JSON files in the dataset directory
for json_file in os.listdir(dataset_dir):
    if json_file.endswith('.json'):
        json_path = os.path.join(dataset_dir, json_file)
        print(len(json_path))

        with open(json_path, 'r') as f:
            data = json.load(f)

        # Process each annotation in the JSON file
        for annotation in data.get('annotations', []):
            # Check if the category_id matches any of the desired category_ids
            label = annotation.get('category_id', None)
            if label in desired_category_ids:
                category_counts[label] += 1

                # If the category_id count exceeds the limit, skip this instance
                if category_counts[label] > category_limits[label]:
                    continue

                label_path = os.path.join(dst_dir, "labels_3")

                if not os.path.exists(label_path):
                    os.makedirs(label_path)

                image_name = data['images'][0]['file_name'].split('.')[0]
                print(f'image_name: {image_name}')

                image_width = data['images'][0]['width']
                image_height = data['images'][0]['height']
                print(f'image_width: {image_width}, image_height: {image_height}')

                bbox = annotation.get('bbox', None)
                print(f'label: {label}, bbox: {bbox}')

                if bbox is not None:
                    x_center, y_center, width, height = convert_to_yolov8_format(bbox=bbox,
                                                                                 image_width=image_width,
                                                                                 image_height=image_height)

                    x_center = format(x_center, '.6f')
                    y_center = format(y_center, '.6f')
                    width = format(width, '.6f')
                    height = format(height, '.6f')

                    arr = list(map(str, [label, x_center, y_center, width, height]))
                    data_line = ' '.join(arr) + '\n'

                    txt_filename = os.path.join(label_path, f"{image_name}.txt")
                    with open(txt_filename, 'a', encoding='UTF-8') as txt_file:
                        txt_file.write(data_line)


print("Category Counts:")
for category_id, count in category_counts.items():
    print(f"Category {category_id}: {count} instances")


In [ ]:
# 디렉토리에 총 몇 개의 .txt 파일이 생겼는지 확인
labels_path = '/content/drive/MyDrive/FinalProject/dataset/labels_3'

txt_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
print("Number of .txt files in the directory:", len(txt_files))

Number of .txt files in the directory: 0


In [ ]:
import os
from collections import Counter

labels_paths = ['/content/drive/MyDrive/FinalProject/dataset/labels']

# Initialize an empty list to store first numbers from all files
first_numbers = []

# Iterate over each labels path
for labels_path in labels_paths:
    # Get all .txt files in the current labels path
    label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]

    # Iterate over each .txt file
    for label_file in label_files:
        label_file_path = os.path.join(labels_path, label_file)

        # Read each line of the current .txt file and get the first number
        with open(label_file_path, 'r') as file:
            for line in file:
                first_number = int(line.split()[0])
                first_numbers.append(first_number)

# Count the occurrences of each first number
frequency_counter = Counter(first_numbers)

# Print the frequency for each class
for class_number, count in frequency_counter.items():
    print(f"Class {class_number}: {count} instances")


Class 8: 1500 instances
Class 9: 1500 instances
Class 14: 1500 instances
Class 13: 1500 instances
Class 6: 1304 instances
Class 12: 1500 instances
Class 16: 1220 instances
Class 11: 1500 instances
Class 5: 1300 instances
Class 3: 1300 instances
Class 4: 1319 instances
Class 2: 1500 instances
Class 7: 1500 instances
Class 15: 944 instances
Class 1: 1500 instances
Class 10: 1500 instances


# Dataset 숫자 맞추기

In [ ]:
import os
import random

def process_txt_files(labels_path, global_limits):
    # Get a list of all .txt files in the directory
    txt_files = [filename for filename in os.listdir(labels_path) if filename.endswith('.txt')]

    # Shuffle the list randomly
    random.shuffle(txt_files)

    # Initialize a dictionary to keep track of counts for each number
    global_counts = {number: 0 for number in global_limits}

    # Iterate over each shuffled .txt file
    for filename in txt_files:
        file_path = os.path.join(labels_path, filename)

        # Read lines from the file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Filter lines based on the global limits for each number
        filtered_lines = []
        for line in lines:
            number = int(line.split()[0])
            if number in global_limits and global_counts[number] < global_limits[number]:
                global_counts[number] += 1
            else:
                filtered_lines.append(line)

        # Write the modified lines back to the file
        with open(file_path, 'w') as file:
            file.writelines(filtered_lines)

        # Remove the file if all lines are removed
        if not filtered_lines:
            os.remove(file_path)

# Set the global limits for each category
global_limits = {1: 367, 2: 778, 3: 2913, 5: 279, 7: 854, 8: 3886,
                 9: 3898, 10: 2147, 11: 44, 12: 508, 13: 2875, 14: 3292}

# Specify the path to the labels directory
labels_path = '/content/drive/MyDrive/FinalProject/dataset/labels'

# Process the .txt files with the given global limits
process_txt_files(labels_path, global_limits)

In [ ]:
#  1. 최종적으로 몇 개의 .txt 파일 남았는지 확인!! => 14,606개
#  2. 남은 .txt 파일에서 instances 몇 개 있는지 개수 확인 => 확인완료
#  3. 너무 차이나면 PM_3 파일에서 채우기 => 215개 채움
#  4. train/valid folder 로 비율맞춰서 옮기기 => 총 14,821 (train: 11,859 / valid: 2,962)

#  5. 학습시작

# Dataset  →  train/valid folder

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# yolo .yaml 파일 형식에 맞추기 위해서 .txt & .jpg 이동
image_source_path = '/content/drive/MyDrive/FinalProject/dataset/img'
label_source_path = '/content/drive/MyDrive/FinalProject/dataset/labels'

split_ratio = 0.8

train_image_dest_path = '/content/drive/MyDrive/FinalProject/train/images'
train_label_dest_path = '/content/drive/MyDrive/FinalProject/train/labels'
valid_image_dest_path = '/content/drive/MyDrive/FinalProject/valid/images'
valid_label_dest_path = '/content/drive/MyDrive/FinalProject/valid/labels'

# original files list
image_files = [f for f in os.listdir(image_source_path) if f.endswith('.jpg')]
train_images, valid_images = train_test_split(image_files,
                                              test_size=1-split_ratio,
                                              random_state=1117)

# move to train directory
for image_file in train_images:
    label_file = image_file.replace('.jpg', '.txt')

    # check the file name between .jpg and .txt
    if os.path.exists(os.path.join(label_source_path, label_file)):
        shutil.move(os.path.join(image_source_path, image_file), os.path.join(train_image_dest_path, image_file))
        shutil.move(os.path.join(label_source_path, label_file), os.path.join(train_label_dest_path, label_file))
        print(f"move to train: {image_file}, {label_file}")

# move to valid directory
for image_file in valid_images:
    label_file = image_file.replace('.jpg', '.txt')

    # check the file name between .jpg and .txt
    if os.path.exists(os.path.join(label_source_path, label_file)):
        shutil.move(os.path.join(image_source_path, image_file), os.path.join(valid_image_dest_path, image_file))
        shutil.move(os.path.join(label_source_path, label_file), os.path.join(valid_label_dest_path, label_file))
        print(f"move to valid: {image_file}, {label_file}")

print("ALL DONE")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
move to train: real_sunny_AM_20220926_104906_sedan_Dong-gu_00091.jpg, real_sunny_AM_20220926_104906_sedan_Dong-gu_00091.txt
move to train: real_sunny_AM_20220913_103810_sedan_Suyeong-gu_02000.jpg, real_sunny_AM_20220913_103810_sedan_Suyeong-gu_02000.txt
move to train: real_sunny_AM_20220913_103810_sedan_Suyeong-gu_02646.jpg, real_sunny_AM_20220913_103810_sedan_Suyeong-gu_02646.txt
move to train: real_sunny_PM_20220705_133934_sedan_Gijang-gun_01897.jpg, real_sunny_PM_20220705_133934_sedan_Gijang-gun_01897.txt
move to train: real_sunny_AM_20220927_110057_sedan_Saha-gu_00034.jpg, real_sunny_AM_20220927_110057_sedan_Saha-gu_00034.txt
move to train: real_sunny_PM_20220714_130344_sedan_Gijang-gun_01030.jpg, real_sunny_PM_20220714_130344_sedan_Gijang-gun_01030.txt
move to train: real_sunny_AM_20220811_103754_sedan_Gangseo-gu_10467.jpg, real_sunny_AM_20220811_103754_sedan_Gangseo-gu_10467.txt
move to train: real_sunny_PM_20220801_130344_sedan_Gijang-gun_0126

In [ ]:
import os

train_image_pth = '/content/drive/MyDrive/FinalProject/train_2/images'
train_label_pth = '/content/drive/MyDrive/FinalProject/train_2/labels'
valid_image_pth = '/content/drive/MyDrive/FinalProject/valid_2/images'
valid_label_pth = '/content/drive/MyDrive/FinalProject/valid_2/labels'

dataset_img_path = '/content/drive/MyDrive/FinalProject/dataset/image/TS_AM_AM_sedan_3'
dataset_labeling_path = '/content/drive/MyDrive/FinalProject/dataset/labels'

train_image_files = [f for f in os.listdir(train_image_pth) if f.endswith('.jpg')]
train_label_files = [f for f in os.listdir(train_label_pth) if f.endswith('.txt')]
valid_image_files = [f for f in os.listdir(valid_image_pth) if f.endswith('.jpg')]
valid_label_files = [f for f in os.listdir(valid_label_pth) if f.endswith('.txt')]
dataset_image_files = [f for f in os.listdir(dataset_img_path) if f.endswith('.jpg')]
dataset_label_files = [f for f in os.listdir(dataset_labeling_path) if f.endswith('.txt')]

print(f'Number of files in the TRAIN directory: image {len(train_image_files)}개, label {len(train_label_files)}개')
print(f'Number of files in the VALID directory: image {len(valid_image_files)}개, label {len(valid_label_files)}개')
print(f'Number of files in the DATASET directory: image {len(dataset_image_files)}개, label {len(dataset_label_files)}개')

Number of files in the TRAIN directory: image 6202개, label 6819개
Number of files in the VALID directory: image 0개, label 0개
Number of files in the DATASET directory: image 6690개, label 0개


In [ ]:
# 동일한 이름의 파일이 train/valid 폴더로 나누어졌는지 확인!
dataset_img_path = '/content/drive/MyDrive/FinalProject/valid/images'
dataset_labeling_path = '/content/drive/MyDrive/FinalProject/valid/labels'

img_files = {f[:-4] for f in os.listdir(dataset_img_path) if f.endswith('.jpg')}
labeling_files = {f[:-4] for f in os.listdir(dataset_labeling_path) if f.endswith('.txt')}

matching_files_count = len(img_files.intersection(labeling_files))

print(f"두 경로에 동일한 파일 이름이 {matching_files_count}개 있습니다.")

두 경로에 동일한 파일 이름이 2915개 있습니다.


In [ ]:
import os

def remove_mismatched_files(img_folder, label_folder):
    img_files = [file for file in os.listdir(img_folder) if file.endswith(".jpg")]
    label_files = [file for file in os.listdir(label_folder) if file.endswith(".txt")]

    # 이미지와 레이블 파일 이름의 차집합을 구함
    mismatched_img_files = set(img_files) - set(label_files)
    mismatched_label_files = set(label_files) - set(img_files)

    # 불일치하는 이미지 파일 삭제
    for file_name in mismatched_img_files:
        file_path = os.path.join(img_folder, file_name)
        os.remove(file_path)
        print(f"이미지 파일 {file_name}을 삭제했습니다.")

    # 불일치하는 레이블 파일 삭제
    for file_name in mismatched_label_files:
        file_path = os.path.join(label_folder, file_name)
        os.remove(file_path)
        print(f"레이블 파일 {file_name}을 삭제했습니다.")

# 주어진 경로에서 파일 삭제 수행
dataset_img_path = '/content/drive/MyDrive/FinalProject/valid_2/images'
dataset_labeling_path = '/content/drive/MyDrive/FinalProject/valid_2/labels'
remove_mismatched_files(dataset_img_path, dataset_labeling_path)

이미지 파일 real_sunny_AM_20220915_104643_sedan_Gangseo-gu_01328.jpg을 삭제했습니다.
이미지 파일 real_sunny_PM_20220905_135430_sedan_Seo-gu_03627.jpg을 삭제했습니다.
이미지 파일 real_wet_AM_20220921_110535_sedan_Dongnae-gu_00526.jpg을 삭제했습니다.
이미지 파일 real_wet_AM_20220831_104304_sedan_Yangsan-si_00635.jpg을 삭제했습니다.
이미지 파일 real_sunny_AM_20220929_104308_sedan_Nam-gu_00710.jpg을 삭제했습니다.
이미지 파일 real_sunny_AM_20220920_103447_sedan_Ulsan-si_00814.jpg을 삭제했습니다.
이미지 파일 real_sunny_PM_20220830_134832_sedan_Gimhae-si_03657.jpg을 삭제했습니다.
이미지 파일 real_wet_AM_20221004_105115_sedan_Geumjeong-gu_00651.jpg을 삭제했습니다.
이미지 파일 real_wet_AM_20220831_104304_sedan_Yangsan-si_00432.jpg을 삭제했습니다.
이미지 파일 real_sunny_AM_20221006_103538_sedan_Buk-gu_01356.jpg을 삭제했습니다.
이미지 파일 real_sunny_AM_20220915_104643_sedan_Gangseo-gu_00264.jpg을 삭제했습니다.
이미지 파일 real_wet_AM_20220922_112216_sedan_Ulsan-si_00965.jpg을 삭제했습니다.
이미지 파일 real_wet_AM_20220831_104304_sedan_Yangsan-si_00462.jpg을 삭제했습니다.
이미지 파일 real_sunny_AM_20220914_111129_sedan_Nam-gu_00042.jpg을 삭제했습니다.
이미지 파일 re

In [ ]:
import os
import shutil

source_path = '/content/drive/MyDrive/FinalProject/train/labels'
destination_path = '/content/drive/MyDrive/FinalProject/dataset/labels'

moved_count = 0
for filename in os.listdir(source_path):
    if filename.endswith('.txt'):
        source_file = os.path.join(source_path, filename)
        destination_file = os.path.join(destination_path, filename)
        shutil.move(source_file, destination_file)
        moved_count += 1

print(f"{moved_count} files moved.")

6391 files moved.


In [ ]:
import os, shutil

# 만약에 파일을 이동해야 한다면?
def move_files(source_folder, destination_folder):
    file_list = os.listdir(source_folder)

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for filename in file_list:
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.move(source_path, destination_path)
    print('ALL DONE')

move_files('/content/drive/MyDrive/FinalProject/train/images', '/content/drive/MyDrive/FinalProject/dataset/img')
move_files('/content/drive/MyDrive/FinalProject/valid/images', '/content/drive/MyDrive/FinalProject/dataset/img')

move_files('/content/drive/MyDrive/FinalProject/train/labels', '/content/drive/MyDrive/FinalProject/dataset/labels')
move_files('/content/drive/MyDrive/FinalProject/valid/labels', '/content/drive/MyDrive/FinalProject/dataset/labels')

ALL DONE
ALL DONE
ALL DONE
ALL DONE


# Dataset pre-processing 완료 & .yaml 파일 생성

In [ ]:
import yaml

# write yaml file with python
data = {"train" : '/content/drive/MyDrive/FinalProject/train/',
        "val" : '/content/drive/MyDrive/FinalProject/valid/',
        "names" : {0 : 'etc', 1 : 'PE드럼 정상', 2 : 'PE드럼 파손', 3 : 'PE방호벽 정상', 4 : 'PE방호벽 파손',
                   5 : 'PE안내봉 정상', 6 : 'PE안내봉 파손', 7 : '라바콘 정상', 8 : '라바콘 파손',
                   9 : '시선유도봉 정상', 10 : '시선유도봉 파손', 11 : '제설함 정상', 12 : '제설함 파손',
                   13 : 'PE입간판 정상', 14 : 'PE입간판 파손', 15 : 'PE휀스 정상', 16 : 'PE휀스 파손'}
        }

with open('/content/drive/MyDrive/FinalProject/data.yaml', 'w', encoding = 'UTF-8') as f :
  yaml.dump(data, f, allow_unicode=True)

# check written file
with open('/content/drive/MyDrive/FinalProject/data.yaml', 'r', encoding = 'UTF-8') as f :
    lines = yaml.safe_load(f)
    print(lines)

# Training using yolov8n.pt

In [ ]:
!nvidia-smi

Thu Jan 25 03:14:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# train
!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/FinalProject/data.yaml epochs=20 imgsz=[1920,1080] rect=True

Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/FinalProject/data.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=[1920, 1080], save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

In [ ]:
# 이어서 학습하기
model = YOLO("/content/drive/MyDrive/FinalProject/runs/detect/train/weights/best.pt")
model.train(resume=True)

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/FinalProject/runs/detect/train/weights/best.pt, data=/content/drive/MyDrive/FinalProject/data.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=1920, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

100%|██████████| 22.2M/22.2M [00:00<00:00, 210MB/s]


TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]  

train: Scanning /content/drive/MyDrive/FinalProject/train/labels.cache... 11868 images, 0 backgrounds, 38 corrupt: 100%|██████████| 11906/11906 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220922_112229_sedan_Yangsan-si_01589.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220922_112229_sedan_Yangsan-si_01589.jpg'
train: WARNING ⚠️ /content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220922_112229_sedan_Yangsan-si_01618.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220922_112229_sedan_Yangsan-si_01618.jpg'
train: WARNING ⚠️ /content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220922_112229_sedan_Yangsan-si_04068.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220922_112229_sedan_Yangsan-si_04068.jpg'
train: WARNING ⚠️ /content/drive/MyDrive/FinalProject/train/images/reproduce_sunny_AM_20220923_10555

...

      19/20      11.3G     0.4107     0.5353     0.8463         13       1920: 100%|██████████| 742/742 [22:54<00:00,  1.85s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 91/91 [04:51<00:00,  3.20s/it]


                   all       2912       4391      0.843      0.913      0.921      0.858

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      11.3G     0.3955     0.5031     0.8387         14       1920: 100%|██████████| 742/742 [22:51<00:00,  1.85s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 91/91 [04:43<00:00,  3.12s/it]


                   all       2912       4391      0.856      0.921      0.927      0.866

6 epochs completed in 3.425 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   3%|▎         | 3/91 [00:04<02:35,  1.76s/it]

In [ ]:
# validation
!yolo task=detect mode=val model=/content/drive/MyDrive/FinalProject/runs/detect/train/weights/best.pt data=/content/drive/MyDrive/FinalProject/data.yaml

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs
100% 22.2M/22.2M [00:00<00:00, 169MB/s]
val: Scanning /content/drive/MyDrive/FinalProject/valid/labels.cache... 2912 images, 0 backgrounds, 3 corrupt: 100% 2915/2915 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/drive/MyDrive/FinalProject/valid/images/reproduce_sunny_AM_20220926_102721_sedan_Yangsan-si_00067.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/MyDrive/FinalProject/valid/images/reproduce_sunny_AM_20220926_102721_sedan_Yangsan-si_00067.jpg'
val: WARNING ⚠️ /content/drive/MyDrive/FinalProject/valid/images/reproduce_sunny_AM_20220926_102721_sedan_Yangsan-si_00535.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/MyDrive/FinalProject/valid/images/reproduce_sunny_AM_20220926_102721_sedan_Yangsan-si_00535.jpg'
val: WARNING ⚠️ /content/drive/MyDrive/FinalProject/vali

In [ ]:
!pip install torchvision

In [ ]:
import torch

# .pt 파일에서 YOLOv8n 모델을 로드합니다.
model_path = '/content/drive/MyDrive/FinalProject/yolov8n.pt'
model = torch.load(model_path, map_location=torch.device('cpu'))

# 모델 아키텍처를 출력합니다.
print(model)

{'epoch': -1, 'best_fitness': None, 'model': DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
 

# evaluation

In [ ]:
# 2차모델 평가!
!yolo task=detect mode=val model=/content/drive/MyDrive/FinalProject/runs/detect/train/weights/best.pt data=/content/drive/MyDrive/FinalProject/data.yaml

Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs
100% 22.2M/22.2M [00:00<00:00, 266MB/s]
val: Scanning /content/drive/MyDrive/FinalProject/dataset/evaluation/labels... 2955 images, 0 backgrounds, 0 corrupt: 100% 2955/2955 [33:58<00:00,  1.45it/s]
val: New cache created: /content/drive/MyDrive/FinalProject/dataset/evaluation/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 185/185 [02:53<00:00,  1.07it/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:467: UserWarning: Glyph 46300 (\N{HANGUL SYLLABLE DEU}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:467: UserWarning: Glyph 47100 (\N{HANGUL SYLLABLE REOM}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultral

In [ ]:
# 1차모델평가!
!yolo task=detect mode=val model=/content/drive/MyDrive/FinalProject/modeling_try_1/detect/train/weights/best.pt data=/content/drive/MyDrive/FinalProject/data.yaml

Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3008963 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/FinalProject/dataset/evaluation/labels.cache... 2955 images, 0 backgrounds, 0 corrupt: 100% 2955/2955 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 185/185 [03:21<00:00,  1.09s/it]
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:467: UserWarning: Glyph 46300 (\N{HANGUL SYLLABLE DEU}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:467: UserWarning: Glyph 47100 (\N{HANGUL SYLLABLE REOM}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:467: UserWarning: Glyph 51221 (\N{HANGUL SYLLABLE JEONG}) missing from current font.
  fig.savefig(save_dir, dp